In [4]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
# from pdf2image import convert_from_bytes
# from PIL import Image, ImageTk
from transformers import pipeline
from PIL import Image, ImageTk


# # Function to display PDF file
# def display_pdf(pdf_file):
#     doc = fitz.open(pdf_file)
#     for page in doc:
#         img = convert_from_bytes(page.get_page_bytes(), dpi=100)[0]
#         img = ImageTk.PhotoImage(img)
#         label = ttk.Label(frame_pdf_viewer, image=img)
#         label.image = img
#         label.pack(padx=10, pady=10)

In [5]:
# Initializing the model
# oracle = pipeline(model="deepset/roberta-base-squad2")
oracle = pipeline("question-answering", model="deepset/roberta-base-squad2")
# mdeberta = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")
whole_word = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")


Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
models = [oracle]
file_name_counter = 0

In [21]:
# Function to convert PDF to text
def pdf_to_text(pdf_file):
    text = ""
    with fitz.open(pdf_file) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Function to find and highlight the relevant texts in the pdf file 
def highlight_pdf(relevant_texts, pdf_file, modified_pdf_file, model_index):
    doc = fitz.open(pdf_file)
    for page in doc:
        for text in relevant_texts:
            if len(text) < 3:
                print('skipping')
                continue
            text_instances = page.search_for(text)
            # print(text_instances)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.update()
    doc.save(modified_pdf_file)
    doc.close()

def display_modified_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    zoom = 0.5  # Adjust the zoom level as needed
    mat = fitz.Matrix(zoom, zoom)

    for page_index in range(doc.page_count):
        page = doc.load_page(page_index)
        pix = page.get_pixmap(matrix=mat)
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        photo_image = ImageTk.PhotoImage(image)
        label = ttk.Label(frame_pdf_viewer, image=photo_image)
        label.image = photo_image
        label.pack(padx=10, pady=10)


In [16]:
# Function to run machine learning model
def run_model(text, prompt,model):
   # Placeholder for your machine learning model
   # Replace this with your actual model code
   all_relevant_texts = model(question= prompt, context=text,
      top_k=5)
   # for line in all_relevant_texts:
      # print(line)
   #  print('the answer given is ', all_relevant_texts['answer'])
   return all_relevant_texts

In [20]:
# Function to handle button click event
def submit_clicked():
    global resume_text, file_name_counter
    pdf_file = entry_pdf.get()
    prompt = entry_prompt.get()

    if pdf_file == "":
        messagebox.showerror("Error", "Please select a PDF file")
        return
    
    if prompt == "":
        messagebox.showerror("Error", "Please enter a prompt")
        return
    
    # Convert PDF to text
    text = pdf_to_text(pdf_file)
    resume_text = text
    # print(text)
    
    # Run models
    for index,model in enumerate(models) :
        model_returns = run_model(text, prompt,model)
        # relevant_texts = ['team','to detail and']
    
        relevant_texts = set()
        for answer in model_returns:
            phrases = answer['answer'].split('\n')
            for phrase in phrases:
                if len(phrase) >= 3:
                    relevant_texts.add(phrase)
                    
        relevant_texts = list(relevant_texts)
        modified_pdf_file = f'Model{index}_{file_name_counter}.pdf'
        highlight_pdf(relevant_texts, pdf_file, modified_pdf_file, index)
        file_name_counter += 1
    # Save modified text to a new PDF file
    # Here you would write code to create a new PDF from modified_text

    # Display the modified PDF in the UI
    for widget in frame_pdf_viewer.winfo_children():
        widget.destroy()
    display_modified_pdf(modified_pdf_file) 
      
    # Display the modified PDF file
    # display_pdf(pdf_file)

In [18]:
# Creating the GUI 

# Create main window
root = tk.Tk()
root.title("PDF Modifier")

# Create style
style = ttk.Style(root)
style.theme_use("clam")

# Create widgets
label_pdf = ttk.Label(root, text="PDF File:")
label_pdf.grid(row=0, column=0, padx=5, pady=5, sticky="w")
entry_pdf = ttk.Entry(root, width=40)
entry_pdf.grid(row=0, column=1, padx=5, pady=5)
button_browse = ttk.Button(root, text="Browse", command=lambda: entry_pdf.insert(tk.END, filedialog.askopenfilename()))
button_browse.grid(row=0, column=2, padx=5, pady=5)

label_prompt = ttk.Label(root, text="Prompt:")
label_prompt.grid(row=1, column=0, padx=5, pady=5, sticky="w")
entry_prompt = ttk.Entry(root, width=40)
entry_prompt.grid(row=1, column=1, padx=5, pady=5)

button_submit = ttk.Button(root, text="Submit", command=submit_clicked)
button_submit.grid(row=2, column=1, padx=5, pady=5)

# PDF Viewer Frame
frame_pdf_viewer = tk.Frame(root)
frame_pdf_viewer.grid(row=3, columnspan=3, padx=10, pady=10, sticky="nsew")
root.rowconfigure(3, weight=1)
root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=1)
root.columnconfigure(2, weight=1)

# Run the Tkinter event loop
root.mainloop()


YOUR NAME
first.last@selu.edu –  985-111-1111
EDUCATION
Southeastern Louisiana University (SLU)
Hammond, LA
Bachelor of Arts in Marketing; Spanish minor
May 2021
Major GPA: 3.50/4.00; Overall GPA: 3.65/4.00
RELEVANT EXPERIENCE
Marketing Intern
Baton Rouge, LA
Louisiana Department of Labor
May 2021 – July 2021
●Screened 40+ applicants for open positions daily, paying attention to detail and clearly communicating job requirements to
potential candidates
●Retrieved unemployment claims, collected confidential customer information, and conducted daily log entries diligently
Chairman of Advertising/Public Relations
Baton Rouge, LA
American Red Cross, Fundraising Committee
January 2020 – May 2020
●Coordinated and supervised a 6-member committee and maximized committee potential by delegating to members’ strengths
●Provided personal outreach to various media outlets which led to first-time coverage from 4 outlets
●Wrote press releases and secured media publicity through radio, paper, and inter

In [19]:
prompt = 'what are some things related to marketing?'
returns = run_model(resume_text, prompt)
# print(returns)

TypeError: run_model() missing 1 required positional argument: 'model'